In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH17-LangGraph-Modules")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH17-LangGraph-Modules


# 병렬 노드 fan-out 및 fan-in

In [2]:
from typing import Annotated, Any
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


# 상태 정의(add_messages 리듀서 사용)
class State(TypedDict):
    aggregate: Annotated[list, add_messages]


# 노드 값 반환 클래스
class ReturnNodeValue:
    # 초기화
    def __init__(self, node_secret: str):
        self._value = node_secret

    # 호출시 상태 업데이트
    def __call__(self, state: State) -> Any:
        print(f"Adding {self._value} to {state['aggregate']}")
        return {"aggregate": [self._value]}

# 상태 그래프 초기화
builder = StateGraph(State)

# 노드 A부터 D까지 생성 및 값 할당
builder.add_node("a", ReturnNodeValue("I'm A"))
builder.add_edge(START, "a")
builder.add_node("b", ReturnNodeValue("I'm B"))
builder.add_node("c", ReturnNodeValue("I'm C"))
builder.add_node("d", ReturnNodeValue("I'm D"))

# 노드 연결
builder.add_edge("a", "b")
builder.add_edge("a", "c")
builder.add_edge("b", "d")
builder.add_edge("c", "d")
builder.add_edge("d", END)

# 그래프 컴파일
graph = builder.compile()

In [3]:
graph.get_graph().print_ascii()

 +-----------+   
 | __start__ |   
 +-----------+   
        *        
        *        
        *        
     +---+       
     | a |       
     +---+       
     *    *      
    *      *     
   *        *    
+---+     +---+  
| b |     | c |  
+---+     +---+  
     *    *      
      *  *       
       **        
     +---+       
     | d |       
     +---+       
        *        
        *        
        *        
  +---------+    
  | __end__ |    
  +---------+    


In [4]:
# 그래프 실행
graph.invoke({"aggregate": []}, {"configurable": {"thread_id": "foo"}})

Adding I'm A to []
Adding I'm B to [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='bf186fda-abb7-4ce0-8777-2c45228f4a26')]
Adding I'm C to [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='bf186fda-abb7-4ce0-8777-2c45228f4a26')]
Adding I'm D to [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='bf186fda-abb7-4ce0-8777-2c45228f4a26'), HumanMessage(content="I'm B", additional_kwargs={}, response_metadata={}, id='0cf91395-262d-49fc-aa7b-318e82fba85c'), HumanMessage(content="I'm C", additional_kwargs={}, response_metadata={}, id='71e951e4-6286-480d-bec4-d144569191bb')]


{'aggregate': [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='bf186fda-abb7-4ce0-8777-2c45228f4a26'),
  HumanMessage(content="I'm B", additional_kwargs={}, response_metadata={}, id='0cf91395-262d-49fc-aa7b-318e82fba85c'),
  HumanMessage(content="I'm C", additional_kwargs={}, response_metadata={}, id='71e951e4-6286-480d-bec4-d144569191bb'),
  HumanMessage(content="I'm D", additional_kwargs={}, response_metadata={}, id='e8b34891-e389-428b-a5ec-6a9e48cdf0c9')]}

# 추가 단계가 있는 병렬 노드의 fan-out 및 fan-in

In [5]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages


# 상태 정의(add_messages 리듀서 사용)
class State(TypedDict):
    aggregate: Annotated[list, add_messages]


# 노드 값 반환 클래스
class ReturnNodeValue:
    # 초기화
    def __init__(self, node_secret: str):
        self._value = node_secret

    # 호출시 상태 업데이트
    def __call__(self, state: State) -> Any:
        print(f"Adding {self._value} to {state['aggregate']}")
        return {"aggregate": [self._value]}


# 상태 그래프 초기화
builder = StateGraph(State)

# 노드 생성 및 연결
builder.add_node("a", ReturnNodeValue("I'm A"))
builder.add_edge(START, "a")
builder.add_node("b1", ReturnNodeValue("I'm B1"))
builder.add_node("b2", ReturnNodeValue("I'm B2"))
builder.add_node("c", ReturnNodeValue("I'm C"))
builder.add_node("d", ReturnNodeValue("I'm D"))
builder.add_edge("a", "b1")
builder.add_edge("a", "c")
builder.add_edge("b1", "b2")
builder.add_edge(["b2", "c"], "d")
builder.add_edge("d", END)

# 그래프 컴파일
graph = builder.compile()

In [6]:
graph.get_graph().print_ascii()

  +-----------+   
  | __start__ |   
  +-----------+   
        *         
        *         
        *         
      +---+       
      | a |       
      +---+       
      *    *      
     *      *     
    *        *    
+----+        *   
| b1 |        *   
+----+        *   
   *          *   
   *          *   
   *          *   
+----+      +---+ 
| b2 |      | c | 
+----+      +---+ 
      *    *      
       *  *       
        **        
      +---+       
      | d |       
      +---+       
        *         
        *         
        *         
   +---------+    
   | __end__ |    
   +---------+    


In [7]:
# 빈 리스트를 사용한 그래프 집계 연산 실행, 모든 데이터에 대한 기본 집계 수행
graph.invoke({"aggregate": []})

Adding I'm A to []
Adding I'm B1 to [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='1f1c81ea-d9b7-4e61-8380-a96881b9490a')]
Adding I'm C to [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='1f1c81ea-d9b7-4e61-8380-a96881b9490a')]
Adding I'm B2 to [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='1f1c81ea-d9b7-4e61-8380-a96881b9490a'), HumanMessage(content="I'm B1", additional_kwargs={}, response_metadata={}, id='1d830827-55d0-4e46-8edd-83d07bf14399'), HumanMessage(content="I'm C", additional_kwargs={}, response_metadata={}, id='5d300fac-eaf4-4b03-bd2b-dd5a32693d38')]
Adding I'm D to [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='1f1c81ea-d9b7-4e61-8380-a96881b9490a'), HumanMessage(content="I'm B1", additional_kwargs={}, response_metadata={}, id='1d830827-55d0-4e46-8edd-83d07bf14399'), HumanMessage(content="I'm C", additional_kwargs={}, response_metadata={}, id='5d300

{'aggregate': [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='1f1c81ea-d9b7-4e61-8380-a96881b9490a'),
  HumanMessage(content="I'm B1", additional_kwargs={}, response_metadata={}, id='1d830827-55d0-4e46-8edd-83d07bf14399'),
  HumanMessage(content="I'm C", additional_kwargs={}, response_metadata={}, id='5d300fac-eaf4-4b03-bd2b-dd5a32693d38'),
  HumanMessage(content="I'm B2", additional_kwargs={}, response_metadata={}, id='72542d76-1012-489c-b1b7-235e93599fca'),
  HumanMessage(content="I'm D", additional_kwargs={}, response_metadata={}, id='4bd805ce-b06c-4f14-938f-0f1d31f386cb')]}

# 조건부 분기(conditional branching)

In [8]:
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langgraph.graph import END, START, StateGraph

# 상태 정의(add_messages 리듀서 사용)
class State(TypedDict):
    aggregate: Annotated[list, add_messages]
    which: str


# 노드별 고유 값을 반환하는 클래스
class ReturnNodeValue:
    def __init__(self, node_secret: str):
        self._value = node_secret

    def __call__(self, state: State) -> Any:
        print(f"Adding {self._value} to {state['aggregate']}")
        return {"aggregate": [self._value]}


# 상태 그래프 초기화
builder = StateGraph(State)
builder.add_node("a", ReturnNodeValue("I'm A"))
builder.add_edge(START, "a")
builder.add_node("b", ReturnNodeValue("I'm B"))
builder.add_node("c", ReturnNodeValue("I'm C"))
builder.add_node("d", ReturnNodeValue("I'm D"))
builder.add_node("e", ReturnNodeValue("I'm E"))


# 상태의 'which' 값에 따른 조건부 라우팅 경로 결정 함수
def route_bc_or_cd(state: State) -> Sequence[str]:
    if state["which"] == "cd":
        return ["c", "d"]
    return ["b", "c"]


# 전체 병렬 처리할 노드 목록
intermediates = ["b", "c", "d"]

builder.add_conditional_edges(
    "a",
    route_bc_or_cd,
    intermediates,
)
for node in intermediates:
    builder.add_edge(node, "e")


# 최종 노드 연결 및 그래프 컴파일
builder.add_edge("e", END)
graph = builder.compile()

In [9]:
graph.get_graph().print_ascii()

      +-----------+        
      | __start__ |        
      +-----------+        
            *              
            *              
            *              
          +---+            
          | a |            
         .+---+..          
       ..   .    ..        
     ..     .      ..      
    .       .        .     
+---+     +---+     +---+  
| b |     | c |     | d |  
+---+**   +---+    *+---+  
       **   *    **        
         ** *  **          
           ** *            
          +---+            
          | e |            
          +---+            
            *              
            *              
            *              
       +---------+         
       | __end__ |         
       +---------+         


In [10]:
# 그래프 실행(which: bc 로 지정)
graph.invoke({"aggregate": [], "which": "bc"})

Adding I'm A to []
Adding I'm B to [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='55ad703f-0f8c-4396-a746-e150ba625804')]
Adding I'm C to [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='55ad703f-0f8c-4396-a746-e150ba625804')]
Adding I'm E to [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='55ad703f-0f8c-4396-a746-e150ba625804'), HumanMessage(content="I'm B", additional_kwargs={}, response_metadata={}, id='f514b90f-2293-4dcd-af03-f7d454383f7d'), HumanMessage(content="I'm C", additional_kwargs={}, response_metadata={}, id='08a4b11a-898f-4aeb-8b4a-2d85b29bc265')]


{'aggregate': [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='55ad703f-0f8c-4396-a746-e150ba625804'),
  HumanMessage(content="I'm B", additional_kwargs={}, response_metadata={}, id='f514b90f-2293-4dcd-af03-f7d454383f7d'),
  HumanMessage(content="I'm C", additional_kwargs={}, response_metadata={}, id='08a4b11a-898f-4aeb-8b4a-2d85b29bc265'),
  HumanMessage(content="I'm E", additional_kwargs={}, response_metadata={}, id='f30fd441-de8f-43c3-bd4d-8ffd98b762e9')],
 'which': 'bc'}

In [11]:
# 그래프 실행(which: cd 로 지정)
graph.invoke({"aggregate": [], "which": "cd"})

Adding I'm A to []
Adding I'm C to [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='c6ed9ce3-f26f-4a63-91ca-dfdc464a3a7d')]
Adding I'm D to [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='c6ed9ce3-f26f-4a63-91ca-dfdc464a3a7d')]
Adding I'm E to [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='c6ed9ce3-f26f-4a63-91ca-dfdc464a3a7d'), HumanMessage(content="I'm C", additional_kwargs={}, response_metadata={}, id='c7cd300b-6a45-4ec8-8481-e2f25ab8a139'), HumanMessage(content="I'm D", additional_kwargs={}, response_metadata={}, id='c69c8af7-b97b-4554-a1c9-1d92332470f9')]


{'aggregate': [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='c6ed9ce3-f26f-4a63-91ca-dfdc464a3a7d'),
  HumanMessage(content="I'm C", additional_kwargs={}, response_metadata={}, id='c7cd300b-6a45-4ec8-8481-e2f25ab8a139'),
  HumanMessage(content="I'm D", additional_kwargs={}, response_metadata={}, id='c69c8af7-b97b-4554-a1c9-1d92332470f9'),
  HumanMessage(content="I'm E", additional_kwargs={}, response_metadata={}, id='581a24a2-0f34-4806-86cd-dc756a76d99e')],
 'which': 'cd'}

# fan-out 값의 신뢰도에 따른 정렬

In [12]:
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages


# 팬아웃 값들의 병합 로직 구현, 빈 리스트 처리 및 리스트 연결 수행
def reduce_fanouts(left, right):
    if left is None:
        left = []
    if not right:
        # 덮어쓰기
        return []
    return left + right


# 상태 관리를 위한 타입 정의, 집계 및 팬아웃 값 저장 구조 설정
class State(TypedDict):
    # add_messages 리듀서 사용
    aggregate: Annotated[list, add_messages]
    fanout_values: Annotated[list, reduce_fanouts]
    which: str


# 그래프 초기화
builder = StateGraph(State)
builder.add_node("a", ReturnNodeValue("I'm A"))
builder.add_edge(START, "a")


# 병렬 노드 값 반환 클래스
class ParallelReturnNodeValue:
    def __init__(
        self,
        node_secret: str,
        reliability: float,
    ):
        self._value = node_secret
        self._reliability = reliability

    # 호출시 상태 업데이트
    def __call__(self, state: State) -> Any:
        print(f"Adding {self._value} to {state['aggregate']} in parallel.")
        return {
            "fanout_values": [
                {
                    "value": [self._value],
                    "reliability": self._reliability,
                }
            ]
        }


# 신뢰도(reliability)가 다른 병렬 노드들 추가
builder.add_node("b", ParallelReturnNodeValue("I'm B", reliability=0.1))
builder.add_node("c", ParallelReturnNodeValue("I'm C", reliability=0.9))
builder.add_node("d", ParallelReturnNodeValue("I'm D", reliability=0.5))


# 팬아웃 값들을 신뢰도 기준으로 정렬하고 최종 집계 수행
def aggregate_fanout_values(state: State) -> Any:
    # 신뢰도 기준 정렬
    ranked_values = sorted(
        state["fanout_values"], key=lambda x: x["reliability"], reverse=True
    )
    print(ranked_values)
    return {
        "aggregate": [x["value"][0] for x in ranked_values] + ["I'm E"],
        "fanout_values": [],
    }


# 집계 노드 추가
builder.add_node("e", aggregate_fanout_values)


# 상태에 따른 조건부 라우팅 로직 구현
def route_bc_or_cd(state: State) -> Sequence[str]:
    if state["which"] == "cd":
        return ["c", "d"]
    return ["b", "c"]


# 중간 노드들 설정 및 조건부 엣지 추가
intermediates = ["b", "c", "d"]
builder.add_conditional_edges("a", route_bc_or_cd, intermediates)

# 중간 노드들과 최종 집계 노드 연결
for node in intermediates:
    builder.add_edge(node, "e")

# 그래프 완성을 위한 최종
graph = builder.compile()

In [13]:
graph.get_graph().print_ascii()

      +-----------+        
      | __start__ |        
      +-----------+        
            *              
            *              
            *              
          +---+            
          | a |            
         .+---+..          
       ..   .    ..        
     ..     .      ..      
    .       .        .     
+---+     +---+     +---+  
| b |     | c |     | d |  
+---+**   +---+    *+---+  
       **   *    **        
         ** *  **          
           ** *            
          +---+            
          | e |            
          +---+            


In [14]:
# 그래프 실행(which: bc 로 지정)
graph.invoke({"aggregate": [], "which": "bc", "fanout_values": []})

Adding I'm A to []
Adding I'm B to [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='48223db7-ebd7-4ff4-a20a-f10f0165b01c')] in parallel.
Adding I'm C to [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='48223db7-ebd7-4ff4-a20a-f10f0165b01c')] in parallel.
[{'value': ["I'm C"], 'reliability': 0.9}, {'value': ["I'm B"], 'reliability': 0.1}]


{'aggregate': [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='48223db7-ebd7-4ff4-a20a-f10f0165b01c'),
  HumanMessage(content="I'm C", additional_kwargs={}, response_metadata={}, id='c67e09f4-6dfa-441f-bc83-b6db9598956c'),
  HumanMessage(content="I'm B", additional_kwargs={}, response_metadata={}, id='dfc7ecda-ca3c-421c-b2b9-fb7e056ecf9d'),
  HumanMessage(content="I'm E", additional_kwargs={}, response_metadata={}, id='fd3a93c6-de9e-4f7b-b4b3-4368796d8296')],
 'fanout_values': [],
 'which': 'bc'}

In [15]:
# 그래프 실행(which: cd 로 지정)
graph.invoke({"aggregate": [], "which": "cd"})

Adding I'm A to []
Adding I'm C to [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='27e5e10a-b44c-44a2-87a2-d4e3b492a035')] in parallel.
Adding I'm D to [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='27e5e10a-b44c-44a2-87a2-d4e3b492a035')] in parallel.
[{'value': ["I'm C"], 'reliability': 0.9}, {'value': ["I'm D"], 'reliability': 0.5}]


{'aggregate': [HumanMessage(content="I'm A", additional_kwargs={}, response_metadata={}, id='27e5e10a-b44c-44a2-87a2-d4e3b492a035'),
  HumanMessage(content="I'm C", additional_kwargs={}, response_metadata={}, id='f74c90ec-0b2c-4586-850f-f0d822523db3'),
  HumanMessage(content="I'm D", additional_kwargs={}, response_metadata={}, id='afcbbda8-c2c2-487c-98f2-0ba911ac16a4'),
  HumanMessage(content="I'm E", additional_kwargs={}, response_metadata={}, id='c265118c-a27c-4f8f-8ef0-620d2f611792')],
 'fanout_values': [],
 'which': 'cd'}